In [1]:
import numpy as np
import cv2
import tensorflow as tf
from tqdm import tqdm
import os
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import InceptionResNetV2
from tensorflow.keras.callbacks import ReduceLROnPlateau, TensorBoard, ModelCheckpoint
import ipywidgets as widgets
import io
from PIL import Image
from IPython.display import display,clear_output

In [3]:
labels = ['Alzheimer MildDemented','Alzheimer ModerateDemented','Alzheimer NonDemented','Alzheimer VeryMildDemented',
          'COVID19 Chest Xray','Lung Opacity','Mild DR','Moderate DR','NORMAL Chest Xray','No  Brain Tumor','No DR',
          'PNEUMONIA Chest Xray','Proliferative DR','Severe DR','actinic keratosis','basal cell carcinoma','benign',
          'dermatofibroma','glioma Brain Tumor','malignant','vascular lesion','squamous cell carcinoma','seborrheic keratosis',
          'pituitary Brain Tumor','pigmented benign keratosis','nevus','meningioma Brain Tumor','melanoma']


In [4]:
len(labels)

28

In [6]:
X=[]
y=[]
image_size = 128
for i in labels:
    folderPath = os.path.join('/kaggle/input/multiclass-disease-classification/Multi Disease Classification','Train',i)
    for j in tqdm(os.listdir(folderPath)):
        img = cv2.imread(os.path.join(folderPath,j))
        img = cv2.resize(img,(image_size, image_size))
        X.append(img)
        y.append(i)
for i in labels:
    folderPath = os.path.join('/kaggle/input/multiclass-disease-classification/Multi Disease Classification','Test',i)
    for j in tqdm(os.listdir(folderPath)):
        img = cv2.imread(os.path.join(folderPath,j))
        img = cv2.resize(img,(image_size,image_size))
        X.append(img)
        y.append(i)
X = np.array(X)
y = np.array(y)

100%|██████████| 16/16 [00:00<00:00, 40.27it/s]


In [7]:
X, y = shuffle(X,y, random_state=101)

In [8]:
X.shape

(53328, 128, 128, 3)

In [9]:
X_train,X_test,y_train,y_test = train_test_split(X,y, test_size=0.2,random_state=101)

In [10]:
y_train_new = []
for i in y_train:
    y_train_new.append(labels.index(i))
y_train = y_train_new
y_train = tf.keras.utils.to_categorical(y_train)


y_test_new = []
for i in y_test:
    y_test_new.append(labels.index(i))
y_test = y_test_new
y_test = tf.keras.utils.to_categorical(y_test)

In [11]:
np.save('X_train',X_train)
np.save('X_test',X_test)
np.save('y_train',y_train)
np.save('y_test',y_test)

In [2]:
X_train=np.load('/kaggle/input/mcdcout/X_train.npy')
X_test=np.load('/kaggle/input/mcdcout/X_test.npy')
y_train=np.load('/kaggle/input/mcdcout/y_train.npy')
y_test=np.load('/kaggle/input/mcdcout/y_test.npy')

In [ ]:

image_size = 128
effnet = InceptionResNetV2(weights='imagenet',include_top=False,input_shape=(image_size,image_size,3))
model = effnet.output
model = tf.keras.layers.GlobalAveragePooling2D()(model)
model= tf.keras.layers.Dense(128,activation='relu')(model)
model = tf.keras.layers.Dropout(rate=0.25)(model)
model = tf.keras.layers.Dense(28,activation='softmax')(model)
model = tf.keras.models.Model(inputs=effnet.input, outputs = model)
model.compile(loss='categorical_crossentropy',optimizer = 'Adam', metrics= ['accuracy'])
tensorboard = TensorBoard(log_dir = 'logs')
checkpoint = ModelCheckpoint("effnet.h5",monitor="val_accuracy",save_best_only=True,mode="auto",verbose=1)
reduce_lr = ReduceLROnPlateau(monitor = 'val_accuracy', factor = 0.3, patience = 10, min_delta = 0.001,mode='auto',verbose=1)
history = model.fit(X_train,y_train,validation_split=0.2, epochs =20, verbose=1, batch_size=128,callbacks=[tensorboard,checkpoint,reduce_lr])

2022-11-29 18:13:30.637264: I tensorflow/core/profiler/lib/profiler_session.cc:131] Profiler session initializing.
2022-11-29 18:13:30.637321: I tensorflow/core/profiler/lib/profiler_session.cc:146] Profiler session started.
2022-11-29 18:13:30.819263: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session tear down.
2022-11-29 18:13:30.819441: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1748] CUPTI activity buffer flushed


Epoch 1/20


2022-11-29 18:13:54.618060: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


  1/234 [..............................] - ETA: 2:06:49 - loss: 3.6066 - accuracy: 0.0156

2022-11-29 18:14:08.218337: I tensorflow/core/profiler/lib/profiler_session.cc:131] Profiler session initializing.
2022-11-29 18:14:08.219017: I tensorflow/core/profiler/lib/profiler_session.cc:146] Profiler session started.


  2/234 [..............................] - ETA: 10:31 - loss: 3.1918 - accuracy: 0.1992  

2022-11-29 18:14:10.700935: I tensorflow/core/profiler/lib/profiler_session.cc:66] Profiler session collecting data.
2022-11-29 18:14:10.711631: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1748] CUPTI activity buffer flushed
2022-11-29 18:14:10.971889: I tensorflow/core/profiler/internal/gpu/cupti_collector.cc:673]  GpuTracer has collected 5012 callback api events and 5009 activity events. 
2022-11-29 18:14:11.128732: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session tear down.
2022-11-29 18:14:11.313528: I tensorflow/core/profiler/rpc/client/save_profile.cc:136] Creating directory: logs/train/plugins/profile/2022_11_29_18_14_11

2022-11-29 18:14:11.414132: I tensorflow/core/profiler/rpc/client/save_profile.cc:142] Dumped gzipped tool data for trace.json.gz to logs/train/plugins/profile/2022_11_29_18_14_11/ae608b548d16.trace.json.gz
2022-11-29 18:14:11.637905: I tensorflow/core/profiler/rpc/client/save_profile.cc:136] Creating directory: logs/train/pl

117/234 [==============>...............] - ETA: 1:21 - loss: 0.8811 - accuracy: 0.6989

In [2]:
pred = model.predict(X_test)
pred = np.argmax(pred,axis=1)
y_test_new = np.argmax(y_test,axis=1)

NameError: name 'model' is not defined

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
print(classification_report(pred,y_test_new))

In [ ]:
np.save('X_train',X_train)
np.save('X_test',X_test)
np.save('y_train',y_train)
np.save('y_test',y_test)